In [1]:
import pandas as pd
import random
import os
import numpy as np
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submission = pd.read_csv('sample_submission.csv')

In [2]:
train.insert(4, 'Height(Inches)', train.apply(lambda x: int(x['Height(Feet)']) * 12 + int(x['Height(Remainder_Inches)']), axis=1))
test.insert(4, 'Height(Inches)', test.apply(lambda x: int(x['Height(Feet)']) * 12 + int(x['Height(Remainder_Inches)']), axis=1))
train = train.drop(['ID', 'Height(Feet)', 'Height(Remainder_Inches)', 'Weight_Status', 'Gender'], axis=1)
test = test.drop(['ID', 'Height(Feet)', 'Height(Remainder_Inches)', 'Weight_Status', 'Gender'], axis=1)

In [3]:
# def remove_outliers(data, threshold=3):
#     z_scores = np.abs((data - np.mean(data)) / np.std(data))
#     return data[z_scores < threshold]

# numeric_features = train.columns[:5]
# for feature in numeric_features:
#     train[feature] = remove_outliers(train[feature])

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7500 entries, 0 to 7499
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   Exercise_Duration    7500 non-null   int64  
 1   Body_Temperature(F)  7500 non-null   float64
 2   BPM                  7500 non-null   int64  
 3   Height(Inches)       7500 non-null   int64  
 4   Weight(lb)           7500 non-null   float64
 5   Age                  7500 non-null   int64  
 6   Calories_Burned      7500 non-null   int64  
dtypes: float64(2), int64(5)
memory usage: 410.3 KB


In [5]:
# ordinal_features = ['Weight_Status', 'Gender']

# for feature in ordinal_features:
#     le = LabelEncoder()
#     le = le.fit(train[feature])
#     train[feature] = le.transform(train[feature])

#     # train데이터에서 존재하지 않았던 값이 test 데이터에 존재할 수도 있습니다.
#     # 따라서 test 데이터를 바로 변형시키지 않고 고윳값을 확인후 test 데이터를 변환합니다.
#     for label in np.unique(test[feature]):
#         if label not in le.classes_:
#             le.classes_ = np.append(le.classes_, label)
#     test[feature] = le.transform(test[feature])

In [6]:
# train_0 = train.loc[train['Weight_Status'] == 0]
# train_1 = train.loc[train['Weight_Status'] == 1]
# train_2 = train.loc[train['Weight_Status'] == 2]
# counts = [len(train_0), len(train_1), len(train_2)]
# train_0 = train_0.sample(n=max(counts), replace=True)
# train_1 = train_1.sample(n=max(counts), replace=True)
# train_2 = train_2.sample(n=max(counts), replace=True)
# train_filled = pd.concat([train_0, train_1, train_2], axis=0)
# train_filled = train_filled.sample(frac=1)

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

def create_regression_model(input_dim):
    model = Sequential()
    model.add(Dense(1000, input_dim=input_dim, activation='relu'))
    model.add(Dense(1000, activation='relu'))
    model.add(Dense(500, activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation=None))
    model.compile(optimizer=Adam(lr=0.0001), loss='mean_squared_logarithmic_error')
    return model

In [21]:
train_x = train.drop(['Calories_Burned'], axis=1)
train_y = train['Calories_Burned']

In [22]:
model = create_regression_model(6)

In [23]:
# 학습 데이터셋을 이용하여 모델을 학습시킵니다.
history = model.fit(train_x, train_y, epochs=50, batch_size=64, validation_split=0.1, verbose=1)

Epoch 1/50
106/106 [==============================] - 1s 5ms/step - loss: 0.8770 - val_loss: 0.4597
Epoch 2/50
106/106 [==============================] - 0s 4ms/step - loss: 0.1399 - val_loss: 0.0419
Epoch 3/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0333 - val_loss: 0.0197
Epoch 4/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0209 - val_loss: 0.0141
Epoch 5/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0178 - val_loss: 0.0129
Epoch 6/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0140 - val_loss: 0.0114
Epoch 7/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0162 - val_loss: 0.0302
Epoch 8/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0134 - val_loss: 0.0089
Epoch 9/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0115 - val_loss: 0.0179
Epoch 10/50
106/106 [==============================] - 0s 4ms/step - loss: 0.0101 - val_loss: 0.0153

In [24]:
preds = model.predict(test)

235/235 [==============================] - 0s 1ms/step


In [25]:
submission['Calories_Burned'] = preds
submission.to_csv('./submit.csv', index = False)